## Unsteady-state Heat Conduction

- Most real situations are time-dependent, i.e. initial temperature distribution may not be in thermal equilibrium with the boundary temperature 
    e.g. batteries generate their own heat upon charge and discharge, but are also thermally heated and cooled with external coolant fluid in EVs 
- Biot number is a dimensionless value that indicates this - internal heat resistance versus surface resistance. Bi << 1 if surface resistance dominates and internal resistance is low. in other words, heat transfer to the system is instantaneously distributed throughout the entire system such that temperature is immediately uniform throughout 

$$\delta^2 T/\delta x^2\ =\ 1/\alpha \delta T/\delta t$$

with B.C.'s $$T(x,0)\ =\ f(x);\ T(0,t)\ =\ 0; T(L,t)\ =\ 0 $$

